# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [145]:
init_aviat_df = pd.read_csv('data/AviationData.csv', encoding = 'latin-1', low_memory = False)

In [146]:
init_aviat_df.head()

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Purpose.of.flight,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date
0,20001218X45444,Accident,SEA87LA080,1948-10-24,"MOOSE CREEK, ID",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,UNK,Cruise,Probable Cause,NaN
1,20001218X45447,Accident,LAX94LA336,1962-07-19,"BRIDGEPORT, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,4.0,0.0,0.0,0.0,UNK,Unknown,Probable Cause,19-09-1996
2,20061025X01555,Accident,NYC07LA005,1974-08-30,"Saltville, VA",United States,36.922223,-81.878056,NaN,NaN,...,Personal,NaN,3.0,NaN,NaN,NaN,IMC,Cruise,Probable Cause,26-02-2007
3,20001218X45448,Accident,LAX96LA321,1977-06-19,"EUREKA, CA",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,12-09-2000
4,20041105X01764,Accident,CHI79FA064,1979-08-02,"Canton, OH",United States,NaN,NaN,NaN,NaN,...,Personal,NaN,1.0,2.0,NaN,0.0,VMC,Approach,Probable Cause,16-04-1980


In [147]:
init_aviat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Investigation.Type      88889 non-null  object 
 2   Accident.Number         88889 non-null  object 
 3   Event.Date              88889 non-null  object 
 4   Location                88837 non-null  object 
 5   Country                 88663 non-null  object 
 6   Latitude                34382 non-null  object 
 7   Longitude               34373 non-null  object 
 8   Airport.Code            50132 non-null  object 
 9   Airport.Name            52704 non-null  object 
 10  Injury.Severity         87889 non-null  object 
 11  Aircraft.damage         85695 non-null  object 
 12  Aircraft.Category       32287 non-null  object 
 13  Registration.Number     87507 non-null  object 
 14  Make                    88826 non-null

In [148]:
init_aviat_df.describe()

,Number.of.Engines,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured
count,82805.000000,77488.000000,76379.000000,76956.000000,82977.000000
mean,1.146585,0.647855,0.279881,0.357061,5.325440
std,0.446510,5.485960,1.544084,2.235625,27.913634
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,0.000000,0.000000,0.000000,2.000000
max,8.000000,349.000000,161.000000,380.000000,699.000000


## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

*The student should be able to identify which columns are relevant -- namely Aircraft.Category, Amateur.Built, and Event.Date*

**Aircraft.Category**
- client is only interested in Airplanes

In [149]:
print(init_aviat_df['Aircraft.Category'].isna().sum())
print(init_aviat_df['Aircraft.Category'].value_counts())

56602
Aircraft.Category
Airplane             27617
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64


In [150]:
init_aviat_df.head()[['Make', 'Aircraft.Category']]

,Make,Aircraft.Category
0,Stinson,NaN
1,Piper,NaN
2,Cessna,NaN
3,Rockwell,NaN
4,Cessna,NaN


In [151]:
# these are all Airplane manufacturers implying that the vast majority of NaNs are Airplanes. 
# Also, given that this is the mode of the known distribution it seems reasonable to impute the column as follows:

init_aviat_df['Aircraft.Category'] = init_aviat_df['Aircraft.Category'].fillna('Airplane')
init_aviat_df['Aircraft.Category'].value_counts()

Aircraft.Category
Airplane             84219
Helicopter            3440
Glider                 508
Balloon                231
Gyrocraft              173
Weight-Shift           161
Powered Parachute       91
Ultralight              30
Unknown                 14
WSFT                     9
Powered-Lift             5
Blimp                    4
UNK                      2
Rocket                   1
ULTR                     1
Name: count, dtype: int64

In [152]:
air_df = init_aviat_df[init_aviat_df['Aircraft.Category'] == 'Airplane']
air_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 84219 entries, 0 to 88888
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                84219 non-null  object 
 1   Investigation.Type      84219 non-null  object 
 2   Accident.Number         84219 non-null  object 
 3   Event.Date              84219 non-null  object 
 4   Location                84169 non-null  object 
 5   Country                 83998 non-null  object 
 6   Latitude                30507 non-null  object 
 7   Longitude               30499 non-null  object 
 8   Airport.Code            48307 non-null  object 
 9   Airport.Name            50802 non-null  object 
 10  Injury.Severity         83289 non-null  object 
 11  Aircraft.damage         81201 non-null  object 
 12  Aircraft.Category       84219 non-null  object 
 13  Registration.Number     82888 non-null  object 
 14  Make                    84159 non-null  obj

**Amateur.Built**
- client is only interested in professional builds

In [153]:
air_df['Amateur.Built'].value_counts()

Amateur.Built
No     76008
Yes     8111
Name: count, dtype: int64

In [154]:
# filter out all amateur builds
air_df = air_df[air_df['Amateur.Built'] == 'No']
air_df['Amateur.Built'].head()

0    No
1    No
2    No
3    No
4    No
Name: Amateur.Built, dtype: object

**Event.Date**
- removing all events older than 40 years ago

In [155]:
air_df = air_df.dropna(subset = ['Publication.Date'])
air_df['Report_Date'] = pd.to_datetime(air_df['Publication.Date'])
air_df = air_df.sort_values(by = ['Report_Date'])
air_df = air_df[air_df['Report_Date'] > '1983']

air_df.head()

C:\Users\Praveen\AppData\Local\Temp\ipykernel_8324\2438261155.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  air_df['Report_Date'] = pd.to_datetime(air_df['Publication.Date'])


,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Air.carrier,Total.Fatal.Injuries,Total.Serious.Injuries,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report_Date
20,20020917X01894,Accident,CHI82FEC08,1982-01-02,"YPSILANTI, MI",United States,NaN,NaN,YIP,WILLOW RUN,...,NaN,0.0,0.0,0.0,1.0,VMC,Takeoff,Probable Cause,02-01-1983,1983-01-02
17,20020917X01656,Accident,ANC82FAG14,1982-01-02,"SKWENTA, AK",United States,NaN,NaN,NaN,NaN,...,NaN,3.0,0.0,0.0,0.0,VMC,Unknown,Probable Cause,02-01-1983,1983-01-02
15,20020917X02117,Accident,FTW82FPG08,1982-01-02,"LITTLE ROCK, AR",United States,NaN,NaN,NaN,NaN,...,NaN,2.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02
14,20020917X02119,Accident,FTW82FPJ10,1982-01-02,"CHICKASHA, OK",United States,NaN,NaN,NaN,NaN,...,NaN,1.0,0.0,0.0,0.0,IMC,Cruise,Probable Cause,02-01-1983,1983-01-02
13,20020917X02134,Accident,FTW82FRA14,1982-01-02,"HEARNE, TX",United States,NaN,NaN,T72,HEARNE MUNICIPAL,...,NaN,1.0,0.0,0.0,0.0,IMC,Takeoff,Probable Cause,02-01-1983,1983-01-02


### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [156]:
# get total number of passenger one each flight
injury_col_names = air_df.columns[air_df.columns.str.contains('njur')][1::]
print(injury_col_names)

air_df[injury_col_names].isna().sum()

Index(['Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured'],
      dtype='object')


Total.Fatal.Injuries       9511
Total.Serious.Injuries    10411
Total.Minor.Injuries       9909
Total.Uninjured            4554
dtype: int64

In [157]:
# many ways to clean. there are some columns that have no data in the injuries stats and should be removed.
air_df[injury_col_names].isna().all(axis = 1).sum()

140

In [158]:
# filters all data that contains NaNs in the injury columns
air_df = air_df[~air_df[injury_col_names].isna().all(axis = 1)]

In [159]:
air_df[injury_col_names].isna().sum()

Total.Fatal.Injuries       9371
Total.Serious.Injuries    10271
Total.Minor.Injuries       9769
Total.Uninjured            4414
dtype: int64

In [160]:
# we will assume for the rest that a blank simply meant that the corresponding data was 0 and was simply not recorded.
# but other valid strategies are dropping all nans, etc.
air_df.loc[:,injury_col_names] = air_df[injury_col_names].fillna(0)
air_df[injury_col_names].isna().sum()

Total.Fatal.Injuries      0
Total.Serious.Injuries    0
Total.Minor.Injuries      0
Total.Uninjured           0
dtype: int64

In [190]:
# the client is interested in the likelihood of fatal/serious injuries given an accident.
# this can be represented as a fraction of fatal+serious injuries over total passengers (injured+uninjured)

# calculate total passenger number
air_df.loc[:,'N_passenger'] = air_df.loc[:,injury_col_names].sum(axis = 1)

# remove any rows where via the total number of passengers has been calculated to be 0
air_df = air_df[air_df['N_passenger'] > 0]
 
# calculate fatal+serious injury fraction

air_df['ser_inj_frac'] = (air_df['Total.Fatal.Injuries'] + air_df['Total.Serious.Injuries'])/air_df['N_passenger']
air_df['ser_inj_frac'].head()

20    0.0
17    1.0
15    1.0
14    1.0
13    1.0
Name: ser_inj_frac, dtype: float64

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [162]:
air_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62451 entries, 20 to 88888
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                62451 non-null  object        
 1   Investigation.Type      62451 non-null  object        
 2   Accident.Number         62451 non-null  object        
 3   Event.Date              62451 non-null  object        
 4   Location                62420 non-null  object        
 5   Country                 62296 non-null  object        
 6   Latitude                25945 non-null  object        
 7   Longitude               25940 non-null  object        
 8   Airport.Code            36821 non-null  object        
 9   Airport.Name            38276 non-null  object        
 10  Injury.Severity         62451 non-null  object        
 11  Aircraft.damage         60437 non-null  object        
 12  Aircraft.Category       62451 non-null  object    

In [163]:
# NaNs and unknowns in Aircraft Damage. Removing them.

In [164]:
print(air_df['Aircraft.damage'].unique())
print(air_df['Aircraft.damage'].value_counts())

['Substantial' 'Destroyed' 'Minor' nan 'Unknown']
Aircraft.damage
Substantial    46048
Destroyed      12494
Minor           1854
Unknown           41
Name: count, dtype: int64


In [165]:
air_df['Aircraft.damage'] = air_df['Aircraft.damage'].replace({'Unknown': np.nan})
air_df.dropna(subset = ['Aircraft.damage'], inplace = True)
air_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60396 entries, 20 to 88784
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                60396 non-null  object        
 1   Investigation.Type      60396 non-null  object        
 2   Accident.Number         60396 non-null  object        
 3   Event.Date              60396 non-null  object        
 4   Location                60368 non-null  object        
 5   Country                 60252 non-null  object        
 6   Latitude                25230 non-null  object        
 7   Longitude               25224 non-null  object        
 8   Airport.Code            35965 non-null  object        
 9   Airport.Name            37414 non-null  object        
 10  Injury.Severity         60396 non-null  object        
 11  Aircraft.damage         60396 non-null  object        
 12  Aircraft.Category       60396 non-null  object    

In [166]:
# created derived column for aircraft destroyed (boolean mask and convert to int with 1 = destroyed)
air_df['is_destroyed'] = (air_df['Aircraft.damage'] == 'Destroyed').astype('int')
air_df['is_destroyed'].head()

20    0
17    1
15    1
14    1
13    1
Name: is_destroyed, dtype: int32

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [167]:
# student should notice that there is a casing problem and that many of the Makes have two few observations for statistical robustness
# casing and string cleaning should be done before count filtering
air_df['Make'].value_counts()

Make
Cessna                17456
Piper                  9435
CESSNA                 4567
Beech                  3439
PIPER                  2668
                      ...  
EVOLUTION TRIKES          1
Barr                      1
BUCKEYE INDUSTRIES        1
Valentin                  1
SCOTT TERRY G             1
Name: count, Length: 1738, dtype: int64

In [168]:
# execute recasing to normalize Make strings
air_df.loc[:,'Make'] = air_df['Make'].str.upper()
air_df['Make'].value_counts()

Make
CESSNA                      22023
PIPER                       12103
BEECH                        4417
BELL                         1348
BOEING                       1157
                            ...  
EVEKTOR-AEROTECHNIK A.S.        1
OPUS MOTORSPORTS LLC            1
CHRISTEN INDUSTRIES INC.        1
ZLIN AVIATION S.R.O.            1
SCOTT TERRY G                   1
Name: count, Length: 1430, dtype: int64

In [169]:
# there are definitely other cleaning tasks that can be executed such as fuzzy string matching 
# if students executes something like this then good...otherwise that's fine

# we list below some valid domain-knowldege and string based merging. the list is not exhaustive.

air_df["Make"] = air_df["Make"].replace(to_replace=["CESSNA AIRCRAFT CO", "CESSNA AIRCRAFT COMPANY", "CESSNA AIRCRAFT", "CESSNA ECTOR", "CESSNA SKYHAWK II", "CESSNA AIRCRAFT CO"], value="CESSNA")
air_df["Make"] = air_df["Make"].replace(to_replace=["PIPER AIRCRAFT INC", "PIPER AIRCRAFT CORPORATION", "PIPER AIRCRAFT", "NEW PIPER AIRCRAFT INC", "'PIPER AIRCRAFT, INC.'", "NEW PIPER", "PIPER AEROSTAR", "PIPER-AEROSTAR", "PIPER PAWNEE", "JETPROP DLX PIPER"], value="PIPER")
air_df["Make"] = air_df["Make"].replace(to_replace=["BEECHCRAFT", "HAWKER BEECHCRAFT", "HAWKER BEECHCRAFT CORP", "HAWKER BEECHCRAFT CORPORATION", "HAWKER BEECHCRAFT CORP.", "BEECH AIRCRAFT CORPORATION", "BEECH AIRCRAFT", "HAWKER BEECH", "HAWKER-BEECHCRAFT", "BEECH AIRCRAFT CO.", "BEECH AIRCRAFT CORP", "BEECHCRAFT CORPORATION", "HAWKER-BEECHCRAFT CORPORATION", "HAWKER", "HAWKER SIDDELEY", "HAWKER AIRCRAFT LTD"], value="BEECH")
air_df["Make"] = air_df["Make"].replace(to_replace=["THE BOEING COMPANY", "BOEING COMPANY", "BOEING STEARMAN", "'BOEING COMPANY, LONG BEACH DIV'", "BOEING-STEARMAN", "BOEING 777-306ER", "BOEING (STEARMAN)", "BOEING COMMERCIAL AIRPLANE GRO", "BOEING OF CANADA/DEHAV DIV", "STEARMAN", "STEARMAN AIRCRAFT"], value="BOEING")
air_df["Make"] = air_df["Make"].replace(to_replace=["MOONEY AIRCRAFT CORP.", "MOONEY AIRPLANE CO INC", "MOONEY INTERNATIONAL CORP", "MOONEY AIRCRAFT CORPORATION", "MOONEY AIRCRAFT CORP", "'MOONEY AIRPLANE COMPANY, INC.'"], value="MOONEY")
air_df["Make"] = air_df["Make"].replace(to_replace=["GRUMMAN ACFT ENG COR-SCHWEIZER", "GRUMMAN AMERICAN AVN. CORP", "GRUMMAN AMERICAN", "GRUMMAN-SCHWEIZER", "GRUMMAN AIRCRAFT ENG CORP", "GRUMMAN ACFT ENG", "GRUMMAN AMERICAN AVIATION", "GRUMMAN SCHWEIZER", "GRUMMAN AIRCRAFT COR-SCHWEIZER", "GRUMMAN AMERICAN AVN. CORP", "GRUMMAN AMERICAN CORPORATION", "GRUMMAN AIRCRAFT", "GRUMMAN ACFT ENG COR", "GRUMMAN AMERICAN AVIATION CORP", "NORTHROP", "GRUMMAN AMERICAN AVN. CORP."], value="GRUMMAN")
air_df["Make"] = air_df["Make"].replace(to_replace=["AIRBUS INDUSTRIE"], value="AIRBUS")
air_df["Make"] = air_df["Make"].replace(to_replace=["MAULE AIRCRAFT CORP"], value="MAULE")
air_df["Make"] = air_df["Make"].replace(to_replace=["AERONCA AIRCRAFT CORPORATION", "AERONCA CHAMPION", "AERONCA CHAMP"], value="AERONCA")

air_df["Make"] = air_df["Make"].replace(to_replace=["AIR TRACTOR", "AIR TRACTOR INC"], value="AIR TRACTOR")
air_df["Make"] = air_df["Make"].replace(to_replace=["CIRRUS", "CIRRUS DESIGN CORP", "CIRRUS DESIGN CORP."], value="CIRRUS")
air_df["Make"] = air_df["Make"].replace(to_replace=["ERCOUPE", "ERCOUPE (ENG & RESEARCH CORP.)"], value="ERCOUPE")
air_df["Make"] = air_df["Make"].replace(to_replace=["AVIAT AIRCRAFT INC", "AVIAT"], value="AVIAT")
air_df["Make"] = air_df["Make"].replace(to_replace=["ROCKWELL", "ROCKWELL INTERNATIONAL"], value="ROCKWELL")



In [170]:
air_df['Make'].value_counts()

Make
CESSNA                            22073
PIPER                             12156
BEECH                              4537
BELL                               1348
GRUMMAN                            1306
                                  ...  
PIAGGIO AERO INDUSTRIES S.P.A.        1
EVEKTOR AEROTECHNIC                   1
EDRA AERONAUTICA                      1
EVEKTOR-AEROTECHNIK A.S.              1
SCOTT TERRY G                         1
Name: count, Length: 1364, dtype: int64

In [171]:
# filtering
filtered_makes = air_df['Make'].value_counts()[air_df['Make'].value_counts() > 50]
filtered_makes[0:25]

Make
CESSNA               22073
PIPER                12156
BEECH                 4537
BELL                  1348
GRUMMAN               1306
BOEING                1240
MOONEY                1139
AIR TRACTOR            851
BELLANCA               820
ROBINSON               602
AERONCA                540
HUGHES                 524
MAULE                  502
CHAMPION               435
SCHWEIZER              431
CIRRUS                 419
STINSON                369
MCDONNELL DOUGLAS      350
LUSCOMBE               349
ROCKWELL               341
AERO COMMANDER         340
NORTH AMERICAN         330
DE HAVILLAND           323
TAYLORCRAFT            316
AVIAT                  214
Name: count, dtype: int64

In [172]:
# filtering makes 
air_df = air_df[air_df['Make'].isin(filtered_makes.index)]

In [173]:
air_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 55587 entries, 20 to 88784
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                55587 non-null  object        
 1   Investigation.Type      55587 non-null  object        
 2   Accident.Number         55587 non-null  object        
 3   Event.Date              55587 non-null  object        
 4   Location                55560 non-null  object        
 5   Country                 55461 non-null  object        
 6   Latitude                22308 non-null  object        
 7   Longitude               22303 non-null  object        
 8   Airport.Code            33126 non-null  object        
 9   Airport.Name            34539 non-null  object        
 10  Injury.Severity         55587 non-null  object        
 11  Aircraft.damage         55587 non-null  object        
 12  Aircraft.Category       55587 non-null  object    

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [174]:
air_df.dropna(subset = ['Model'], inplace = True)

In [175]:
# as an example both Beech and Aero_commander have a model 100
air_df.groupby(['Model', 'Make']).count().loc['100']

,Event.Id,Investigation.Type,Accident.Number,Event.Date,Location,Country,Latitude,Longitude,Airport.Code,Airport.Name,...,Total.Minor.Injuries,Total.Uninjured,Weather.Condition,Broad.phase.of.flight,Report.Status,Publication.Date,Report_Date,N_passenger,ser_inj_frac,is_destroyed
Make,,,,,,,,,,,,,,,,,,,,,
AERO COMMANDER,33,33,33,33,33,33,16,16,16,22,...,33,33,33,25,31,33,33,33,33,33
BEECH,11,11,11,11,11,11,4,4,7,7,...,11,11,11,7,10,11,11,11,11,11
ROCKWELL,1,1,1,1,1,1,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [176]:
# one possibility is to use multi-indexing. another possibility is to merge make_model into a single column

In [177]:
air_df['make_model'] = air_df['Make'] + '_' + air_df['Model'].str.upper()
air_df['make_model'].head()

20      CESSNA_152
17      CESSNA_180
15        BEECH_19
14        NAVION_A
13    CESSNA_R172K
Name: make_model, dtype: object

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [178]:
print(air_df['Engine.Type'].value_counts())
air_df['Engine.Type'] = air_df['Engine.Type'].replace({'Unknown': np.nan, 'UNK': np.nan}) 
# dont want any 'Unknown' categories computed in our statistics
# replacing them with NaNs is appropriate and ensures statistics calculated over what we care about
# you dont need to impute these NaNs to compute relevant statistics.

# clean out single value as one can't use this to make statistical claims
filtered_enginetype = air_df['Engine.Type'].value_counts()[air_df['Engine.Type'].value_counts() > 1]
air_df = air_df[air_df['Engine.Type'].isin(filtered_enginetype.index)]
air_df['Engine.Type'].value_counts()

Engine.Type
Reciprocating      46597
Turbo Prop          2434
Turbo Shaft         1553
Turbo Fan           1091
Unknown              875
Turbo Jet            301
Geared Turbofan        1
UNK                    1
Name: count, dtype: int64


Engine.Type
Reciprocating    46597
Turbo Prop        2434
Turbo Shaft       1553
Turbo Fan         1091
Turbo Jet          301
Name: count, dtype: int64

In [179]:
# converting unknowns to NaNs
print(air_df['Weather.Condition'].unique())
air_df['Weather.Condition'] = air_df['Weather.Condition'].replace({'UNK': np.nan, 'Unk': np.nan})
print(air_df['Weather.Condition'].value_counts())

['VMC' 'IMC' 'UNK' nan 'Unk']
Weather.Condition
VMC    47094
IMC     4006
Name: count, dtype: int64


In [180]:
# having zero engines for an airplane doesnt make sense. Since its only two examples, remove corresponding entries.
print(air_df['Number.of.Engines'].value_counts())
air_df = air_df[air_df['Number.of.Engines'] > 0.0]
print(air_df['Number.of.Engines'].unique())

Number.of.Engines
1.0    44405
2.0     6751
4.0      221
3.0      219
0.0        2
Name: count, dtype: int64
[1. 2. 3. 4.]


In [181]:
# converting unknowns to NaNs

print(air_df['Purpose.of.flight'].unique())
air_df['Purpose.of.flight'].replace({'Unknown': np.nan}, inplace = True)

['Instructional' 'Personal' 'Ferry' 'Business' 'Unknown'
 'Aerial Observation' 'Executive/corporate' 'Aerial Application'
 'Public Aircraft' 'Other Work Use' 'Positioning' nan 'Flight Test'
 'Air Drop' 'Air Race/show' 'Skydiving' 'Public Aircraft - State'
 'Glider Tow' 'External Load' 'Firefighting' 'Banner Tow'
 'Public Aircraft - Local' 'Public Aircraft - Federal' 'Air Race show'
 'PUBS']


C:\Users\Praveen\AppData\Local\Temp\ipykernel_8324\2983482517.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  air_df['Purpose.of.flight'].replace({'Unknown': np.nan}, inplace = True)


In [182]:
# converting unknowns and Other to NaNs

print(air_df['Broad.phase.of.flight'].unique())
air_df['Broad.phase.of.flight'] = air_df['Broad.phase.of.flight'].replace({'Unknown': np.nan, 'Other': np.nan})
print(air_df['Broad.phase.of.flight'].value_counts())

['Takeoff' 'Unknown' 'Cruise' 'Taxi' 'Approach' 'Landing' 'Maneuvering'
 'Descent' 'Climb' 'Go-around' 'Other' 'Standing' nan]
Broad.phase.of.flight
Landing        10462
Takeoff         7870
Cruise          6592
Maneuvering     4890
Approach        4075
Taxi            1343
Climb           1280
Descent         1132
Go-around        966
Standing         520
Name: count, dtype: int64


### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [183]:
# inspect data
air_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51596 entries, 20 to 88632
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Id                51596 non-null  object        
 1   Investigation.Type      51596 non-null  object        
 2   Accident.Number         51596 non-null  object        
 3   Event.Date              51596 non-null  object        
 4   Location                51585 non-null  object        
 5   Country                 51474 non-null  object        
 6   Latitude                20137 non-null  object        
 7   Longitude               20132 non-null  object        
 8   Airport.Code            31674 non-null  object        
 9   Airport.Name            33106 non-null  object        
 10  Injury.Severity         51596 non-null  object        
 11  Aircraft.damage         51596 non-null  object        
 12  Aircraft.Category       51596 non-null  object    

In [184]:
# some valid column removals are FAR.Description, Air.carrier, Schedule

air_df = air_df.drop(columns = ['FAR.Description', 'Air.carrier', 'Schedule'])

# column list should no longer retain above columns
air_df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type',
       'Purpose.of.flight', 'Total.Fatal.Injuries', 'Total.Serious.Injuries',
       'Total.Minor.Injuries', 'Total.Uninjured', 'Weather.Condition',
       'Broad.phase.of.flight', 'Report.Status', 'Publication.Date',
       'Report_Date', 'N_passenger', 'ser_inj_frac', 'is_destroyed',
       'make_model'],
      dtype='object')

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [185]:
# student should save data as csv to appropriate folder
air_df.to_csv('data/air_cleaned.csv', index = False)